In [2]:
%run model.ipynb

Computing gene knockout strategies using the OptGene algorithm

### Preparations

Basing algorithms upon the surfactin reaction which is based in the fatty acid that is most utilized. Looking at the reaction stoichiometry again:

In [3]:
print(model.reactions.SP_1)

SP_1: 3hddcoa_c + asp__L_c + atp_c + glu__L_c + 4 leu__L_c + nadph_c + val__L_c --> adp_c + nadp_c + surfactin_c


Finding the other reaction metabolite names that are needed:

In [4]:
model.reactions.query("biomass","name") #Biomass reaction name

[<Reaction BIOMASS_BS_10 at 0x1cf96ce0dc0>]

In [5]:
model.metabolites.get_by_id("glc__D_e") #Glucose metabolite name

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x01cf9671be50
Formula,C6H12O6
Compartment,e
In 6 reaction(s),"GLCt2, GLCpts, BG_MBDG, EX_glc__D_e, BG_MADG, BG_CELLB"


Changing objective to surfactin

In [6]:
model.objective = model.reactions.SP_1

Import the packages which are necessary for OptGene:

In [7]:
import cameo #after pip install cameo in terminal

In [8]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [9]:
from cameo import models

### OptGene Algorithm

Running the OptGene algorithm to search for gene or reaction knockouts:

In [10]:
optgene = OptGene(model)

In [11]:
result_optgene = optgene.run(target="SP_1", #Reaction
                     biomass="BIOMASS_BS_10", #Reaction
                     substrate="glc__D_e", #Metabolite
                     max_evaluations=20000, max_knockouts=3, plot=False)

Starting optimization at Wed, 24 Nov 2021 21:40:47


HBox()

Finished after 00:12:03


C:\Users\Sandie\anaconda3\envs\Cell factory\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [12]:
result_optgene

HBox()

,reactions,genes,size,fva_min,fva_max,target_flux,biomass_flux,yield,fitness
0,"(CDGPT_BS,)","((BSU16920,),)",1,0.0,0.074928,0.0,0.023593,0.0,0.0
1,"(UGT2_BS, UGT_BS, UGT1_BS)","((BSU21920,),)",1,0.0,0.075374,0.0,0.023593,0.0,0.0
2,"(UGT2_BS, CDGPT_BS, UGT_BS, UGT1_BS)","((BSU21920, BSU16920),)",2,0.0,0.075685,0.0,0.023593,0.0,0.0


No optimization suggestions were obtained from Optgene. 

Double checking this result by manually knocking out this reaction and finding surfactin yield:

In [31]:
with model as mutant0:
    mutant0.genes.BSU16920.knock_out()
    print("Flux:",mutant0.slim_optimize())
    surfactin_production = model.optimize().objective_value
    print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
    print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

Flux: 0.07492830302601322
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07492830302601322
Theoretical max. yield [mmol-surfactin / mmol-sucrose]: 0.04407547236824307


In [29]:
with model as mutant1:
    mutant1.genes.BSU21920.knock_out()
    print("Flux:",mutant1.slim_optimize())
    surfactin_production = model.optimize().objective_value
    print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
    print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

Flux: 0.07537374203987006
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07537374203987006
Theoretical max. yield [mmol-surfactin / mmol-sucrose]: 0.04433749531757063


In [30]:
with model as mutant2:
    mutant2.genes.BSU21920.knock_out()
    mutant2.genes.BSU16920.knock_out()
    print("Flux:", mutant2.slim_optimize())
    surfactin_production = model.optimize().objective_value
    print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
    print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_glc__D_e.flux))

Flux: 0.07568534919274368
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07568534919274368
Theoretical max. yield [mmol-surfactin / mmol-sucrose]: 0.0445207936427904


In [25]:
with model:
    model.reactions.FDH.bounds = 0, 0
    print("IPDDI's bounds inside the with statement")
    print(model.reactions.FDH.lower_bound, model.reactions.FDH.bounds)
    print('Mutant growth rate [mmol gDW^-1 h^-1]: ', model.optimize().objective_value)
    with model:
        model.objective = model.reactions.SP_1
        surfactin_production = model.optimize().objective_value
        print("Max. surfactin production [mmol gDW^-1 h^-1]:", surfactin_production)
        print("Theoretical max. yield [mmol-surfactin / mmol-sucrose]:", surfactin_production / (-1*model.reactions.EX_sucr_e.flux))


IPDDI's bounds inside the with statement
0 (0, 0)
Mutant growth rate [mmol gDW^-1 h^-1]:  0.07461282448997686
Max. surfactin production [mmol gDW^-1 h^-1]: 0.07461282448997686


ZeroDivisionError: float division by zero

It seems that the OptKnock algorithm has suggested a few knockoutS that leads to an increase in yield.